### Introduction
- 브라운더스트 구글 플레이스토어 내 모든 리뷰 데이터 수집
- 페이지 스크롤 내리기와 더보기 버튼 무한 실행으로 전체 리뷰 viewing
- selenium 과 beautifulsoup 으로 리뷰 내용과 평점에 접근
- Dataframe 형태로 저장
- (Selenium webdriver로 페이지 무한 스크롤 시 request 속도 저하 발생)

#### Import libraries

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.get('https://play.google.com/store/apps/details?id=com.neowizgames.game.browndust.srpg.gamfs&showAllReviews=true') #접속할 Review URL page

#### Web page control

In [ ]:
time_now = time.time()
flag=0 
while 1:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        loadMore=driver.find_element_by_xpath("//*[contains(@class,'U26fgb O0WRkf oG5Srb C0oVfc n9lfJ')]").click()
    except:
        time.sleep(1)
        flag=flag+1
        if flag >= 10: 
            break
    else:
        flag=0
print("{} seconds spent to viewing all reviews".format(time.time()-time_now))

In [4]:
table_dict = {}
date_list = []
contents_list = []
stars_list = []

reviews=driver.find_elements_by_xpath("//*[@jsname='fk8dgd']//div[@class='d15Mdf bAhLNe']")

#### Rating

In [5]:
star_cnt = 0
for review in reviews: #text로 설명된 rating
    soup=BeautifulSoup(review.get_attribute("innerHTML"),"lxml")
    ratings=soup.find('div',role='img').get('aria-label')
    stars_list.append(ratings)
    star_cnt += 1

print("{}개의 review를 수집합니다.".format(star_cnt))

19439개의 review를 수집합니다.


#### Review text

In [6]:
for i in range(1,star_cnt+1): #review text
    review_txt = driver.find_elements_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div/div['+ str(i) + ']/div/div[2]/div[2]/span[1]')[0].text
    contents_list.append(review_txt)
    if i%100==0:
        print("{}th review text".format(i))

100th review text
200th review text
300th review text
400th review text
500th review text
600th review text
700th review text
800th review text
900th review text
1000th review text
1100th review text
1200th review text
1300th review text
1400th review text
1500th review text
1600th review text
1700th review text
1800th review text
1900th review text
2000th review text
2100th review text
2200th review text
2300th review text
2400th review text
2500th review text
2600th review text
2700th review text
2800th review text
2900th review text
3000th review text
3100th review text
3200th review text
3300th review text
3400th review text
3500th review text
3600th review text
3700th review text
3800th review text
3900th review text
4000th review text
4100th review text
4200th review text
4300th review text
4400th review text
4500th review text
4600th review text
4700th review text
4800th review text
4900th review text
5000th review text
5100th review text
5200th review text
5300th review text
54

#### Review date

In [7]:
for i in range(1,star_cnt+1): #review date
    review_date = driver.find_elements_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div/div['+ str(i) + ']/div/div[2]/div[1]/div[1]/div/span[2]')[0].text
    date_list.append(review_date)
    if i%100==0:
        print("{}th review date".format(i))

100th review date
200th review date
300th review date
400th review date
500th review date
600th review date
700th review date
800th review date
900th review date
1000th review date
1100th review date
1200th review date
1300th review date
1400th review date
1500th review date
1600th review date
1700th review date
1800th review date
1900th review date
2000th review date
2100th review date
2200th review date
2300th review date
2400th review date
2500th review date
2600th review date
2700th review date
2800th review date
2900th review date
3000th review date
3100th review date
3200th review date
3300th review date
3400th review date
3500th review date
3600th review date
3700th review date
3800th review date
3900th review date
4000th review date
4100th review date
4200th review date
4300th review date
4400th review date
4500th review date
4600th review date
4700th review date
4800th review date
4900th review date
5000th review date
5100th review date
5200th review date
5300th review date
54

#### Saving Data Frame 

In [8]:
table_dict['time'] = date_list
table_dict['review_txt'] = contents_list
table_dict['stars_txt'] = stars_list

print('\n','\n','\n')

review_df = pd.DataFrame.from_dict(table_dict)
review_df['time'] = review_df.loc[:,'time'].str.replace(" ","")
review_df['time'] = pd.to_datetime(review_df.loc[:,'time'], format = '%Y년%m월%d일')
print('가장 최근 작성 review 날짜 : {} '.format(review_df['time'].max()))
print('가장 오래전 작성 review 날짜 : {} '.format(review_df['time'].min()))

review_df['ratings'] = review_df['stars_txt'].str[10]
review_df['ratings'] = review_df['ratings'].astype(int)

print('review 별 평점 분포', '\n', review_df['ratings'].value_counts())
print('review 별 평점 분포 %', '\n', review_df['ratings'].value_counts(normalize = True))
    
review_df.to_csv('review_crwl_browndust.csv', index = False)

driver.close()


 
 

가장 최근 작성 review 날짜 : 2019-05-23 00:00:00 
가장 오래전 작성 review 날짜 : 2017-04-24 00:00:00 
review 별 평점 분포 
 5    13678
4     3420
1     1208
3      866
2      267
Name: ratings, dtype: int64
review 별 평점 분포 % 
 5    0.703637
4    0.175935
1    0.062143
3    0.044550
2    0.013735
Name: ratings, dtype: float64
